In [1]:
import tensorflow.keras as keras
import numpy as np
from scipy.io import loadmat
from my_classes import DataGenerator
import matplotlib.pyplot as plt
import glob
import pickle
%matplotlib inline

model_from_json = keras.models.model_from_json

types = ['kmno4', 'set1', 'set2', 'set5', 'set9']
ModelCheckpoint=keras.callbacks.ModelCheckpoint
Sequential=keras.models.Sequential
Dense=keras.layers.Dense
Flatten=keras.layers.Flatten
Dropout=keras.layers.Dropout
np.random.seed(7)

/home/jayathu/anaconda3/envs/ml_35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


The main purpose of creating a data generator is to avoid unnecessary usage of RAM when using larger data sets. `my_classes.py` contains such a generator.

Creating `partition` dictionary

Loading `partition` dictionary

In [2]:
with open('nnet2_partition.pickle', 'rb') as handle:
    partition=pickle.load(handle)

The saved dictionary is 90MB as shown below:

In [4]:
!ls -lh nnet2_partition.pickle

-rw-rw-r--. 1 jayathu jayathu 90M Sep  8 09:18 nnet2_partition.pickle


In [3]:
params = {'dim': (16,32),
          'batch_size': 64,
          'n_classes': 5,
          'n_channels': 1,
          'shuffle': True}

In [4]:
# Generators
training_generator = DataGenerator(partition['train'], [], **params)
validation_generator = DataGenerator(partition['val'], [], **params)
testing_generator = DataGenerator(partition['test'], [], **params)

# Model fitting

In [5]:
model = Sequential()
model.add(Flatten(input_shape=(*params['dim'],1)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(.25))
model.add(Dense(5, activation='softmax'))

In [6]:
# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [9]:
filepath="models/nnet4/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=4,
                    epochs=5,
                    callbacks=callbacks_list,
                    verbose=0)


Epoch 00001: val_acc improved from -inf to 0.90808, saving model to models/nnet4/weights-improvement-01-0.91.hdf5


# Saving model

In [7]:
# serialize model to JSON
model_json = model.to_json()
with open("models/nnet4/model.json", "w") as json_file:
    json_file.write(model_json)

# Evaluate model with test data

In [22]:
json_file = open('models/nnet4/model.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights('models/nnet4/weights-improvement-03-0.92.hdf5')
# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Check if model is alright.

In [25]:
# evaluate the model
scores=model.evaluate_generator(testing_generator, steps=None, max_queue_size=10, workers=4, use_multiprocessing=1, verbose=1)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

21608/21608 [==============================] - 1296s 60ms/step

acc: 92.08%
